In [ ]:
# The bins in our ND flux estimation and in our ND flux uncertainties don't match, so we redistribute the events
# in our NuWro samples event by event - each event in an off-axis position sample is randomly assigned to the bins
# it overlaps with. This means we assume that the change in the nominal flux between
# adjacent off-axis bins is negligible.
# Prerequisites:
# 1) A .csv file with energy bin center values, extracted from the TH2Jagged, called BinCenters.csv
# 2) A .csv file with energy bin width values, extracted from the TH2Jagged, called BinWidths.csv

In [ ]:
from bisect import bisect as bisect
import numpy as np
from csv import reader
import numpy.random as rnd
import ROOT
import os
import time

%jsroot on

In [ ]:
centers = []

with open('BinCenters.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        centers.append(1000*np.array(row).astype(float)[1:])
centers = centers[1:-1]

widths = []

with open('BinWidths.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        widths.append(1000*np.array(row).astype(float)[1:])
widths = [np.array(widths[i])/2 for i in range(len(widths))][1:-1]

def probs(low_edge,high_edge):
    prob1 = 1
    prob2 = 0
    if np.floor(high_edge)-np.floor(low_edge) == 1:
        prob1 = np.ceil(low_edge)-low_edge
        prob2 = high_edge-np.floor(high_edge)
    norm = prob1+prob2
    return [prob1/norm,prob2/norm]

floors = [[int(np.floor(0.574*i)),int(np.floor(0.574*i))+1] for i in range(58)]

EbE_bins = np.array([np.where(np.random.rand(int(1e6))<probs(0.574*i,0.574*(i+1))[0],floors[i][0],floors[i][1]) for i in range(58)])

np.save('EbE_bins.npy',EbE_bins)